# Talk to Your Data: AI-Powered Conservation Analysis

A hands-on workshop using AI to explore and visualize conservation data.

**What you'll do:**
- Describe your analytical questions in plain English
- Gemini generates the code to answer them
- Build custom visualizations through conversation
- No programming experience required

**Data source:** Miradi Share sample project (exported to SQLite).

**Project: Eastern Bay Village (aka Miradi Marine Example)**

Working with the residents of Eastern Bay Village to help manage and conserve the marine resources of their traditional fishing grounds.

Source:
https://www.miradishare.org/ux/project/fos-miradimarineprogr-2019-00007

---

## Step 1: Load the Data

First, let's download and load our sample conservation project data.

In [ ]:
# Download the sample data from GitHub
!wget -q https://raw.githubusercontent.com/fosonline/miradi-data-exploration/main/sample_project.sqlite -O project.sqlite
print('Data downloaded!')

In [ ]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('project.sqlite')

# Let's see what tables we have
tables = pd.read_sql("""
    SELECT name FROM sqlite_master 
    WHERE type='table' AND name != 'extraction_metadata'
    ORDER BY name
""", conn)

print("Available data tables:")
for t in tables['name']:
    count = pd.read_sql(f'SELECT COUNT(*) as n FROM "{t}"', conn)['n'][0]
    print(f"  - {t}: {count} records")

## Step 2: Explore the Data

Let's look at what's in the key tables. Ask Gemini:
- "What columns are in the biodiversity_targets table?"
- "Show me a sample of the indicators data"
- "What's the relationship between measurements and indicators?"

In [ ]:
# Load key tables as pandas DataFrames for easy exploration
targets = pd.read_sql('SELECT * FROM biodiversity_targets', conn)
strategies = pd.read_sql('SELECT * FROM strategies', conn)
threats = pd.read_sql('SELECT * FROM direct_threats', conn)
indicators = pd.read_sql('SELECT * FROM indicators', conn)
measurements = pd.read_sql('SELECT * FROM measurements', conn)

print(f"Loaded {len(targets)} targets, {len(strategies)} strategies, {len(threats)} threats")
print(f"Loaded {len(indicators)} indicators with {len(measurements)} measurements")

In [ ]:
# Look at the targets (conservation focal species/ecosystems)
targets[['identifier', 'name', 'calculated_viability_status', 'calculated_viability_future_status']].head(10)

## Exercise 1: Visualize Target Status

In the Gemini sidebar, try typing:

> "Create a bar chart showing the viability status of all biodiversity targets"

**Tips:**
- If the code doesn't work, tell Gemini the error
- Ask it to change colors, labels, or chart type
- Try: *"Make it a pie chart instead"*

In [ ]:
# AI-generated code will go here
# Try: "Create a bar chart showing the viability status of all biodiversity targets"

import matplotlib.pyplot as plt

# Example visualization (ask AI to improve or customize)
if 'calculated_viability_status' in targets.columns:
    rating_counts = targets['calculated_viability_status'].value_counts()
    plt.figure(figsize=(10, 6))
    rating_counts.plot(kind='bar', color='steelblue')
    plt.title('Biodiversity Target Viability Status')
    plt.xlabel('Status')
    plt.ylabel('Number of Targets')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Exercise 2: Network Diagrams

Ask Gemini to visualize relationships:

> "Create a network diagram showing how threats relate to targets"

> "Show me which strategies address which threats"

**Tip:** Tell Gemini to look at [Appendix A: Miradi Data Model Reference](#Appendix-A:-Miradi-Data-Model-Reference) for help with data relationships.

In [ ]:
# Install networkx if needed
!pip install -q networkx

import networkx as nx
import json

# AI-generated network visualization code goes here
# NOTE: Relationships are stored as JSON arrays. See "Appendix A: Miradi Data
# Model Reference" at the bottom of this notebook for join patterns and table schemas.

## Exercise 3: Time Series

The measurements table has monitoring data from 2012-2025.

> "Plot the measurement values over time for coral reef indicators"

> "Are any indicators showing decline?"

> "Compare trends across all targets"

**Tip:** To join indicators with measurements, see the join patterns in [Appendix A: Miradi Data Model Reference](#Appendix-A:-Miradi-Data-Model-Reference).

In [ ]:
# Look at measurement data structure
# NOTE: To join indicators with measurements, use the LIKE pattern described
# in "Appendix A: Miradi Data Model Reference" at the bottom of this notebook.
if len(measurements) > 0:
    print("Measurement columns:")
    print(measurements.columns.tolist())
    print("\nSample measurements:")
    display(measurements.head())

## Exercise 4: Maps

The project has location data (latitude, longitude).

> "Show the project location on a map"

> "Create a map with the project boundary"

> "Pull in the target footprints as well"

In [ ]:
# Load project location data
projects = pd.read_sql('SELECT * FROM projects', conn)

if len(projects) > 0:
    project = projects.iloc[0]
    print(f"Project: {project['title']}")
    print(f"Countries: {project.get('countries', 'N/A')}")
    if 'footprint_latitude' in project:
        print(f"Location: {project['footprint_latitude']}, {project['footprint_longitude']}")

In [ ]:
# Install folium for interactive maps
!pip install -q folium

import folium

# AI-generated map code goes here

## Exercise 5: Your Questions

Try your own analysis questions:

- *"Which targets don't have any indicators?"* (gap analysis)
- *"What's the budget distribution across strategies?"*
- *"Create a summary dashboard of project health"*
- *"Write a SQL query to count indicators per target"*

**Discuss with your team: what questions would you ask of YOUR project data?**

**Tip:** See [Appendix A: Miradi Data Model Reference](#Appendix-A:-Miradi-Data-Model-Reference) for the full list of tables, fields, and join patterns.

In [ ]:
# Your custom analysis code goes here
# Ask Gemini to help you explore!
# NOTE: See "Appendix A: Miradi Data Model Reference" at the bottom of this
# notebook for table schemas, join patterns, and rating scales.

---

## Key Takeaways

1. **Describe what you want** -- Gemini generates the code
2. **Iterate** -- refine by describing what to change
3. **Domain expertise matters** -- AI doesn't know your project, you do
4. **This works with any structured data** -- not just Miradi

**Next steps:**
- Try with your own project data (export from Miradi Share)
- Explore other AI tools (Claude, ChatGPT, GitHub Copilot)
- Apply to SMART, EarthRanger, or any CSV/database

In [ ]:
# Close the database connection when done
conn.close()
print("Thanks for participating!")

---

## Appendix A: Miradi Data Model Reference

Miradi is a conservation planning tool based on the [Open Standards for the Practice of Conservation](https://conservationstandards.org/). The data model reflects the adaptive management cycle:

1. **Assess** - Identify biodiversity targets, threats, and situation analysis
2. **Plan** - Define goals, strategies, objectives, and activities
3. **Implement** - Track work plans, resources, and expenses
4. **Analyze & Adapt** - Monitor indicators, measure progress, learn and adjust

Run the next cell to see the Entity Relationship Diagram.

In [ ]:
# Render the Miradi data model ERD
import base64
from IPython.display import Image, display, Markdown

mermaid_code = '''
erDiagram
    projects ||--o{ biodiversity_targets : contains
    projects ||--o{ human_wellbeing_targets : contains
    projects ||--o{ direct_threats : contains
    projects ||--o{ strategies : contains
    projects ||--o{ conceptual_models : contains
    projects ||--o{ results_chains : contains

    biodiversity_targets ||--o{ goals : has
    biodiversity_targets ||--o{ indicators : monitored_by
    biodiversity_targets ||--o{ stresses : affected_by

    direct_threats }o--o{ biodiversity_targets : threatens
    direct_threats ||--o{ contributing_factors : caused_by

    strategies ||--o{ activities : contains
    strategies ||--o{ objectives : achieves
    strategies ||--o{ indicators : monitored_by

    activities ||--o{ tasks : contains

    indicators ||--o{ measurements : has
    indicators ||--o{ future_statuses : has

    results_chains ||--o{ intermediate_results : contains
    results_chains ||--o{ threat_reduction_results : contains
'''

# Render via mermaid.ink (no dependencies needed)
encoded = base64.urlsafe_b64encode(mermaid_code.encode()).decode()
display(Markdown("### Entity Relationship Diagram"))
display(Image(url=f"https://mermaid.ink/img/{encoded}"))

### Important: How to Join Tables

**AI Assistant Note:** This database uses JSON arrays to store relationships. When generating SQL queries that join tables, use the pattern shown below.

#### Pattern 1: Parent owns children (JSON array on parent)

The parent table has a column containing a JSON array of child URIs. Use `LIKE` with the child's `resource_identifier`:

```sql
-- Find measurements for an indicator
SELECT i.name as indicator_name, m.measurement_date, m.value, m.trend
FROM indicators i
JOIN measurements m ON i.measurements LIKE '%' || m.resource_identifier || '%'
WHERE i.name LIKE '%coral%'
ORDER BY m.measurement_date;

-- Find activities for a strategy  
SELECT s.name as strategy_name, a.name as activity_name
FROM strategies s
JOIN activities a ON s.activities LIKE '%' || a.resource_identifier || '%';

-- Find goals for a target
SELECT bt.name as target_name, g.name as goal_name
FROM biodiversity_targets bt
JOIN goals g ON bt.goals LIKE '%' || g.resource_identifier || '%';
```

#### Pattern 2: Child references parent (single URI on child)

Some child tables have a column with a single parent URI. Use `LIKE` matching:

```sql
-- Find activities and their parent strategy
SELECT a.name as activity_name, s.name as strategy_name
FROM activities a
JOIN strategies s ON a.strategy LIKE '%' || s.resource_identifier || '%';

-- Find tasks and their parent activity
SELECT t.name as task_name, a.name as activity_name
FROM tasks t
JOIN activities a ON t.activity LIKE '%' || a.resource_identifier || '%';
```

#### Key Relationship Fields

| Parent Table | Relationship Field | Child Table |
|--------------|-------------------|-------------|
| `indicators` | `measurements` | `measurements` |
| `indicators` | `future_statuses` | `future_statuses` |
| `strategies` | `activities` | `activities` |
| `strategies` | `monitoring_activities` | `monitoring_activities` |
| `activities` | `tasks` | `tasks` |
| `biodiversity_targets` | `goals` | `goals` |
| `biodiversity_targets` | `indicators` | `indicators` |
| `biodiversity_targets` | `stresses` | `stresses` |
| `conceptual_models` | `strategies` | `strategies` |
| `conceptual_models` | `direct_threats` | `direct_threats` |
| `results_chains` | `intermediate_results` | `intermediate_results` |

### Core Tables

**Project Context**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `projects` | Top-level project with geospatial footprint | `title`, `footprint_latitude`, `footprint_longitude`, `countries` |
| `project_summaries` | Overall project status ratings | `overall_viability_status`, `overall_threat_rating` |
| `project_scopes` | Scope description (biological, cultural, social) | `vision`, `scope`, `biological_description` |
| `project_plans` | Work plan configuration and budget | `start_date`, `expected_end_date`, `currency_type` |
| `project_resources` | Staff and team members | `first_name`, `last_name`, `position` |
| `project_organizations` | Stakeholder organizations | `organization_name`, `roles_description` |

**Conservation Targets**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `biodiversity_targets` | Species, ecosystems, or processes to conserve | `name`, `calculated_viability_status`, `calculated_threat_rating` |
| `human_wellbeing_targets` | Human benefits linked to conservation | `name`, `calculated_viability_status` |
| `key_ecological_attributes` | Measurable attributes defining target health | `name`, `key_ecological_attribute_type` |
| `stresses` | Degraded conditions caused by threats | `name`, `severity_rating`, `scope_rating` |

**Threats & Factors**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `direct_threats` | Human activities that negatively affect targets | `name`, `calculated_threat_rating` |
| `contributing_factors` | Indirect factors that drive direct threats | `name` |
| `biophysical_factors` | Natural/physical factors in the situation model | `name` |
| `project_threat_ratings` | Threat-target rating matrix | `threat_ratings` (JSON) |

**Strategies & Actions**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `strategies` | Interventions to address threats | `name`, `status`, `feasibility_rating`, `impact_rating` |
| `activities` | Specific actions within a strategy | `name`, `strategy` (parent URI) |
| `monitoring_activities` | Activities focused on monitoring | `name`, `strategy` (parent URI) |
| `tasks` | Sub-tasks within activities | `name`, `activity` (parent URI) |

**Monitoring & Measurement**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `indicators` | Measurable variables to track progress | `name`, `unit`, `priority_rating`, `measurements` (JSON array) |
| `measurements` | Actual data points collected | `measurement_date`, `value`, `rating`, `trend` |
| `future_statuses` | Projected future values | `status_date`, `rating`, `summary` |

**Results Chain Elements**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `intermediate_results` | Expected outcomes between strategies and targets | `name` |
| `threat_reduction_results` | Expected reductions in threat severity | `name`, `related_direct_threat` (parent URI) |
| `results_chains` | Theory of change diagrams | `name`, `strategies`, `intermediate_results` (JSON arrays) |
| `conceptual_models` | Situation analysis diagrams | `name`, `biodiversity_targets`, `direct_threats` (JSON arrays) |

**Work Planning**
| Table | Description | Key Fields |
|-------|-------------|------------|
| `funding_sources` | Sources of project funding | `name`, `code` |
| `resource_assignments` | Staff allocations | `project_resource`, `funding_source` (URIs) |
| `expense_assignments` | Budget allocations | `funding_source` (URI), `expenses` (JSON) |

### Rating Scales

| Type | Values (best to worst) |
|------|------------------------|
| Viability Status | `Very Good` > `Good` > `Fair` > `Poor` |
| Threat Rating | `Very High` > `High` > `Medium` > `Low` |
| Trend | `Strong Increase`, `Mild Increase`, `Flat`, `Mild Decrease`, `Strong Decrease` |